In [ ]:
!pip install pycoingecko

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycoingecko import CoinGeckoAPI
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
import random

# Crie uma lista de 10 IDs aleatórios de criptomoedas
cg = CoinGeckoAPI()
coins = cg.get_coins_list()
coin_ids = random.sample([coin['id'] for coin in coins], 10)

# Defina o intervalo de tempo para os dados históricos (6 semanas atrás até agora)
end_date = datetime.now()
start_date = end_date - timedelta(weeks=6)

# Recupere dados históricos para cada criptomoeda
dfs = []
for coin_id in coin_ids:
    data = cg.get_coin_market_chart_range_by_id(coin_id, 'usd', start_date.strftime('%s'), end_date.strftime('%s'))
    prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
    prices['timestamp'] = pd.to_datetime(prices['timestamp'], unit='ms')
    prices.set_index('timestamp', inplace=True)
    prices = prices.resample('W').last()
    prices.columns = [coin_id]
    dfs.append(prices)

# Combine os dados em uma única tabela
table = pd.concat(dfs, axis=1)

# Use a regressão linear para estimar os valores das próximas duas semanas
model = LinearRegression()
X = pd.Series(range(len(table)))
X = X.values.reshape(-1, 1)
y = table.iloc[:, 0].values
model.fit(X, y)
X_new = pd.Series([len(table), len(table)+1])
X_new = X_new.values.reshape(-1, 1)
y_new = model.predict(X_new)

# Aplica estilo à tabela
table_style = table.style.format('{:,.2f}').set_table_styles([{    'selector': 'th',    'props': [('background-color', 'gray'), ('color', 'white')]
}, {
    'selector': 'td',
    'props': [('border', '1px solid black')]
}])

# Display 
table_style


,artery,polygen,weta-vr,cashera,bananatok,betterment-digital,virtual-reality-glasses,conflux-token,beefy-escrowed-fantom,geist-usdc
timestamp,,,,,,,,,,
2023-01-22 00:00:00,0.01,0.00,nan,0.00,0.00,0.00,0.00,0.03,0.32,1.00
2023-01-29 00:00:00,0.01,0.00,nan,0.00,0.00,0.00,0.00,0.06,0.40,1.00
2023-02-05 00:00:00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.06,0.44,1.00
2023-02-12 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.05,0.35,1.00
2023-02-19 00:00:00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.21,0.41,1.00
2023-02-26 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.23,0.37,1.00
2023-03-05 00:00:00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.19,0.32,1.00
